# Analyse the differences between control run and 10% increasing in all Arctic runoffs with all tiles
# (Polar Stereographic Projection)

### Set-up and infrastructure

In [2]:
# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys

# identify user's home directory
user_home_dir = expanduser('~')

# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

import botocore  
import boto3
import os
import glob
import cmocean
import re
from boto3.session import Session
import cmocean
from collections import Counter
from dask.distributed import Client
import datetime
import fsspec
from gc import get_referents
import json
import numpy as np
from pathlib import Path
from pprint import pprint
import requests
import pandas as pd
import s3fs
import sys
from sys import getsizeof
import time as time
from types import ModuleType, FunctionType
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import zarr
import matplotlib.cm as cm
import matplotlib.path as mpath
import imageio.v2 as imageio
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [3]:
# Use this for the netcdf files stored on an s3 bucket
def get_credentials(use_earthdata=False):
    """
    This routine automatically pulls your EDL crediential from .netrc file and use it to obtain an AWS S3 credential 
    through a PO.DAAC service accessible at https://archive.podaac.earthdata.nasa.gov/s3credentials.
    From the PO.DAAC Github (https://podaac.github.io/tutorials/external/July_2022_Earthdata_Webinar.html).
    
    Returns:
    =======
    
    credentials: a dictionary with AWS secret_key, access_key, and token
    """
    # NASA EarthData hosts ECCO V4r4 fileds
    if use_earthdata == False:
        session = boto3.Session()
        credentials_b3 = session.get_credentials()
        creds_b3 = credentials_b3.get_frozen_credentials()
        
        credentials = dict()
        credentials['secretAccessKey'] = credentials_b3.secret_key
        credentials['accessKeyId'] = credentials_b3.access_key
        credentials['sessionToken'] = credentials_b3.token

    # A 'public' AWS s3 bucket hosts V4r5 fields (they will eventually move to PO.DAAC)
    else:
        credentials = requests.get('https://archive.podaac.earthdata.nasa.gov/s3credentials').json()
    
    return credentials
    

In [4]:
def init_S3FileSystem(use_earthdata=False, requester_pays=True):
    """
    This routine automatically creates an 's3 file system' object and credentials dictionary.
    The s3 file system needs to be initialized with the special aws credentials.
    
    Returns:
    =======
    
    s3: an AWS S3 filesystem, 
    credentials: a dictionary with AWS secret_key, access_key, and token

    """
    credentials = get_credentials(use_earthdata=use_earthdata)

    if use_earthdata:
        requester_pays = False
        
    s3 = s3fs.S3FileSystem(requester_pays=requester_pays,
                           anon=False,
                           key=credentials['accessKeyId'],
                           secret=credentials['secretAccessKey'], 
                           token=credentials['sessionToken'])
    
    return s3, credentials

In [5]:
# function for determining the memory footprint of an object

# ... from https://stackoverflow.com/questions/449560/how-do-i-determine-the-size-of-an-object-in-python

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType

def getsize(obj):
    """
    This routine returns the in-memory size of an python object
    
    Returns:
    =======
    
    size: size of object & members.
    """
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [6]:
from datetime import timedelta, datetime

def date_to_iter_number(date,seconds_per_iter = 3600):
    total_seconds = (date-datetime(1992,1,1)).total_seconds()
    iter_number = total_seconds/seconds_per_iter
    return(iter_number)

def iter_number_to_date(iter_number,seconds_per_iter=3600):
    total_seconds = iter_number*seconds_per_iter
    date = datetime(1992,1,1) + timedelta(seconds=total_seconds)
    return(date)

In [7]:
from dask.distributed import Client

#  connect to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:33875")
client.ncores
client.restart()

In [8]:
# local path to monthly-mean native grid datasets
ecco_v4r5_mon_mean_native_dir = Path('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/')

# list sub-directories (one per dataset)
ecco_v4r5_mon_mean_native_dataset_paths = np.sort(list(ecco_v4r5_mon_mean_native_dir.glob('*')))

for i, d in enumerate(ecco_v4r5_mon_mean_native_dataset_paths):
    print(str(i).zfill(3),d)

000 /efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc


In [9]:
# ---------------------------------------------------------------
# Check november dates -- can change to your year as required
# ---------------------------------------------------------------
from datetime import timedelta, datetime

oct_start = date_to_iter_number(datetime(2019,10,1))
dec_end = date_to_iter_number(datetime(2019,12,31))

In [10]:
# --------------------------------------------------
# Take the geometry file for grid information
# --------------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

## Loading Salinity gifs for run with ten percent increase in total runoff minus control run

In [10]:
# -------------------------------------------------
# Load in three months of control run -- November
# -------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

salt_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename);
            salt_test = np.where(geom.hFacC == 1, salt_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list_base.append(salt_DA);

# Concatenate all valid files
salt_base_OND = xr.concat(salt_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000243288.data
load_binary_array: data a

In [11]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in total runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

salt_DA_list_tenpercent = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename);
            salt_test = np.where(geom.hFacC == 1, salt_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list_tenpercent.append(salt_DA);

# Concatenate all valid files
salt_tenpercent_OND = xr.concat(salt_DA_list_tenpercent, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean/SALT_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean/SALT_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean/SALT_daily_mean.0000243288.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_co

In [12]:
# If it's 4D (e.g., time, k, j, i), select surface level and specific tile
data_baserun = salt_base_OND.isel(k=0)
data_tenpercentrun = salt_tenpercent_OND.isel(k=0)

# Assume geom is your ECCO dataset including grid info
lon_corners = geom.XG
lat_corners = geom.YG

# Temporary directory for frames
tmp_dir = 'tmp_frames_salt'
os.makedirs(tmp_dir, exist_ok=True)
filenames = []

# Loop through each time step
for t in range(len(data_baserun.time)):
    fig = plt.figure(figsize=(8, 6), dpi=90)
    fig.suptitle("1.1x total runoff - control run")
    cmap = matplotlib.colormaps.get_cmap('bwr')
    
    # Difference between scenarios
    diff = data_tenpercentrun.isel(time=t) - data_baserun.isel(time=t)
    
    # Mask land or missing data using hFacC (ocean fraction = 0 → land)
    mask = geom.hFacC.isel(k=0)
    mask = mask.reindex_like(diff)  # Reindex mask to have the same coordinates as diff
    diff = diff.where(mask != 0, np.nan)

    fig, ax, CS, cbar, *_ = ecco.plot_proj_to_latlon_grid(
        lon_corners, lat_corners, 
        diff, 
        projection_type='stereo',
        show_colorbar=True,
        dx=1, dy=1, cmin=-0.5, cmax=0.5,
        lat_lim=40, cmap=cmap,
        return_fig=True
    )
    
    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.gridlines()
    ax.coastlines()
    cbar.set_label('Salinity [PSU]')  # Add units
    
    # Set zoomed region
    #zoom_extent = [-170, -120, 66, 80]  # Zoom on Mackenzie
    zoom_extent = [-180, 180, 60, 90]  # Zoom on Gloabl Arctic
    ax.set_extent(zoom_extent, ccrs.PlateCarree())

    # Clip plot with circular boundary around the pole if you want
    # theta = np.linspace(0, 2*np.pi, 100)
    # center, radius = [0.5, 0.5], 0.5
    # verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    # circle = mpath.Path(verts * radius + center)
    # ax.set_boundary(circle, transform=ax.transAxes)  # Comment if you want regional zoom on Mackenzie runoff

    # Title: timestamp from the data
    ax.set_title(str(data_baserun.time[t].values)[:10])

    # Save current frame
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    filenames.append(filename)

# Build the animated GIF
gif_filename = 'salt_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif'
#gif_filename = 'salt_tenper_minus_base_OND2019_polar_ster_proj_mack_tiles.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove temporary frames
# for filename in filenames:
#     os.remove(filename)
# os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

GIF saved to salt_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif


## Loading Sea Ice Area gifs for run with ten percent increase in total runoff minus control run

In [10]:
# -------------------------------------------------
# Load in three months of control run -- November 
# -------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean'
pattern = os.path.join(input_dir, 'SIarea_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siarea_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siarea_test = ecco.read_llc_to_tiles(input_dir, filename);
            siarea_test = np.where(geom.hFacC == 1, siarea_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siarea_DA = xr.DataArray(
                siarea_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siarea_DA_list_base.append(siarea_DA);

# Concatenate all valid files
siarea_base_OND = xr.concat(siarea_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean/SIarea_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean/SIarea_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean/SIarea_daily_mean.0000243288.data
load_binary_a

In [11]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in total runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent/diags/SIarea_daily_mean'
pattern = os.path.join(input_dir, 'SIarea_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siarea_DA_list_tenpercent = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siarea_test = ecco.read_llc_to_tiles(input_dir, filename);
            siarea_test = np.where(geom.hFacC == 1, siarea_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siarea_DA = xr.DataArray(
                siarea_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siarea_DA_list_tenpercent.append(siarea_DA);

# Concatenate all valid files
siarea_tenpercent_OND = xr.concat(siarea_DA_list_tenpercent, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SIarea_daily_mean/SIarea_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SIarea_daily_mean/SIarea_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SIarea_daily_mean/SIarea_daily_mean.0000243288.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type

In [12]:
# If it's 4D (e.g., time, k, j, i), select surface level and specific tile
data_baserun = siarea_base_OND.isel(k=0)
data_tenpercentrun = siarea_tenpercent_OND.isel(k=0)

# Assume geom is your ECCO dataset including grid info
lon_corners = geom.XG
lat_corners = geom.YG

# Temporary directory for frames
tmp_dir = 'tmp_frames_siarea'
os.makedirs(tmp_dir, exist_ok=True)
filenames = []

# Loop through each time step
for t in range(len(data_baserun.time)):
    fig = plt.figure(figsize=(8, 6), dpi=90)
    fig.suptitle("1.1x total runoff - control run")
    cmap = matplotlib.colormaps.get_cmap('PiYG')
    
    # Difference between scenarios
    diff = data_tenpercentrun.isel(time=t) - data_baserun.isel(time=t)
    
    # Mask land or missing data using hFacC (ocean fraction = 0 → land)
    mask = geom.hFacC.isel(k=0)
    mask = mask.reindex_like(diff)  # Reindex mask to have the same coordinates as diff
    diff = diff.where(mask != 0, np.nan)

    fig, ax, CS, cbar, *_ = ecco.plot_proj_to_latlon_grid(
        lon_corners, lat_corners, 
        diff, 
        projection_type='stereo',
        show_colorbar=True,
        dx=1, dy=1, cmin=-0.05, cmax=0.05,
        lat_lim=40, cmap=cmap,
        return_fig=True
    )
    
    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.gridlines()
    ax.coastlines()
    cbar.set_label('Sea Ice Area [-]')  # Add units
    
    # Set zoomed region
    #zoom_extent = [-170, -120, 66, 80]  # Zoom on Mackenzie
    zoom_extent = [-180, 180, 60, 90]  # Zoom on Gloabl Arctic
    ax.set_extent(zoom_extent, ccrs.PlateCarree())

    # Clip plot with circular boundary around the pole if you want
    # theta = np.linspace(0, 2*np.pi, 100)
    # center, radius = [0.5, 0.5], 0.5
    # verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    # circle = mpath.Path(verts * radius + center)
    # ax.set_boundary(circle, transform=ax.transAxes)  # Comment if you want regional zoom on Mackenzie runoff

    # Title: timestamp from the data
    ax.set_title(str(data_baserun.time[t].values)[:10])

    # Save current frame
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    filenames.append(filename)

# Build the animated GIF
gif_filename = 'siarea_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif'
#gif_filename = 'siarea_tenper_minus_base_OND2019_polar_ster_proj_mack_tiles.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove temporary frames
# for filename in filenames:
#     os.remove(filename)
# os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

GIF saved to siarea_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif


## Loading Sea Ice Effective Thickness gifs for run with ten percent increase in total runoff minus control run

In [11]:
# -------------------------------------------------
# Load in three months of control run -- November 
# -------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean'
pattern = os.path.join(input_dir, 'SIheff_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siheff_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siheff_test = ecco.read_llc_to_tiles(input_dir, filename);
            siheff_test = np.where(geom.hFacC == 1, siheff_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siheff_DA = xr.DataArray(
                siheff_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siheff_DA_list_base.append(siheff_DA);

# Concatenate all valid files
siheff_base_OND = xr.concat(siheff_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean/SIheff_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean/SIheff_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean/SIheff_daily_mean.0000243288.data
load_binary_a

In [12]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in total runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent/diags/SIheff_daily_mean'
pattern = os.path.join(input_dir, 'SIheff_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siheff_DA_list_tenpercent = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siheff_test = ecco.read_llc_to_tiles(input_dir, filename);
            siheff_test = np.where(geom.hFacC == 1, siheff_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siheff_DA = xr.DataArray(
                siheff_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siheff_DA_list_tenpercent.append(siheff_DA);

# Concatenate all valid files
siheff_tenpercent_OND = xr.concat(siheff_DA_list_tenpercent, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SIheff_daily_mean/SIheff_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SIheff_daily_mean/SIheff_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SIheff_daily_mean/SIheff_daily_mean.0000243288.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type

In [13]:
# If it's 4D (e.g., time, k, j, i), select surface level and specific tile
data_baserun = siheff_base_OND.isel(k=0)
data_tenpercentrun = siheff_tenpercent_OND.isel(k=0)

# Assume geom is your ECCO dataset including grid info
lon_corners = geom.XG
lat_corners = geom.YG

# Temporary directory for frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)
filenames = []

# Loop through each time step
for t in range(len(data_baserun.time)):
    fig = plt.figure(figsize=(8, 6), dpi=90)
    fig.suptitle("1.1x total runoff - control run")
    cmap = matplotlib.colormaps.get_cmap('BrBG')
    
    # Difference between scenarios
    diff = data_tenpercentrun.isel(time=t) - data_baserun.isel(time=t)
    
    # Mask land or missing data using hFacC (ocean fraction = 0 → land)
    mask = geom.hFacC.isel(k=0)
    mask = mask.reindex_like(diff)  # Reindex mask to have the same coordinates as diff
    diff = diff.where(mask != 0, np.nan)

    fig, ax, CS, cbar, *_ = ecco.plot_proj_to_latlon_grid(
        lon_corners, lat_corners, 
        diff, 
        projection_type='stereo',
        show_colorbar=True,
        dx=1, dy=1, cmin=-0.1, cmax=0.1,
        lat_lim=40, cmap=cmap,
        return_fig=True
    )
    
    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.gridlines()
    ax.coastlines()
    cbar.set_label('Sea Ice Effective Thickness [m]')  # Add units
    
    # Set zoomed region
    #zoom_extent = [-170, -120, 66, 80]  # Zoom on Mackenzie
    zoom_extent = [-180, 180, 60, 90]  # Zoom on Gloabl Arctic
    ax.set_extent(zoom_extent, ccrs.PlateCarree())

    # Clip plot with circular boundary around the pole if you want
    # theta = np.linspace(0, 2*np.pi, 100)
    # center, radius = [0.5, 0.5], 0.5
    # verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    # circle = mpath.Path(verts * radius + center)
    # ax.set_boundary(circle, transform=ax.transAxes)  # Comment if you want regional zoom on Mackenzie runoff

    # Title: timestamp from the data
    ax.set_title(str(data_baserun.time[t].values)[:10])

    # Save current frame
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    filenames.append(filename)

# Build the animated GIF
gif_filename = 'siheff_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif'
#gif_filename = 'siheff_tenper_minus_base_OND2019_polar_ster_proj_mack_tiles.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove temporary frames
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

GIF saved to siheff_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif


## Loading Temperature gifs for run with ten percent increase in total runoff minus control run

In [14]:
# -------------------------------------------------
# Load in three months of control run -- November 
# -------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

theta_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list_base.append(theta_DA);

# Concatenate all valid files
theta_base_OND = xr.concat(theta_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000243288.data
load_binary_array: 

In [15]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in total runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

theta_DA_list_tenpercent = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list_tenpercent.append(theta_DA);

# Concatenate all valid files
theta_tenpercent_OND = xr.concat(theta_DA_list_tenpercent, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/THETA_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/THETA_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/THETA_daily_mean.0000243288.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4


In [16]:
# If it's 4D (e.g., time, k, j, i), select surface level and specific tile
data_baserun = theta_base_OND.isel(k=0)
data_tenpercentrun = theta_tenpercent_OND.isel(k=0)

# Assume geom is your ECCO dataset including grid info
lon_corners = geom.XG
lat_corners = geom.YG

# Temporary directory for frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)
filenames = []

# Loop through each time step
for t in range(len(data_baserun.time)):
    fig = plt.figure(figsize=(8, 6), dpi=90)
    fig.suptitle("1.1x total runoff - control run")
    cmap = matplotlib.colormaps.get_cmap('RdBu')
    
    # Difference between scenarios
    diff = data_tenpercentrun.isel(time=t) - data_baserun.isel(time=t)
    
    # Mask land or missing data using hFacC (ocean fraction = 0 → land)
    mask = geom.hFacC.isel(k=0)
    mask = mask.reindex_like(diff)  # Reindex mask to have the same coordinates as diff
    diff = diff.where(mask != 0, np.nan)

    fig, ax, CS, cbar, *_ = ecco.plot_proj_to_latlon_grid(
        lon_corners, lat_corners, 
        diff, 
        projection_type='stereo',
        show_colorbar=True,
        dx=1, dy=1, cmin=-0.1, cmax=0.1,
        lat_lim=40, cmap=cmap,
        return_fig=True
    )
    
    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.gridlines()
    ax.coastlines()
    cbar.set_label('Potential Temperature [°C]')  # Add units
    
    # Set zoomed region
    #zoom_extent = [-170, -120, 66, 80]  # Zoom on Mackenzie
    zoom_extent = [-180, 180, 60, 90]  # Zoom on Gloabl Arctic
    ax.set_extent(zoom_extent, ccrs.PlateCarree())

    # Clip plot with circular boundary around the pole if you want
    # theta = np.linspace(0, 2*np.pi, 100)
    # center, radius = [0.5, 0.5], 0.5
    # verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    # circle = mpath.Path(verts * radius + center)
    # ax.set_boundary(circle, transform=ax.transAxes)  # Comment if you want regional zoom on Mackenzie runoff

    # Title: timestamp from the data
    ax.set_title(str(data_baserun.time[t].values)[:10])

    # Save current frame
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    filenames.append(filename)

# Build the animated GIF
gif_filename = 'theta_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif'
#gif_filename = 'theta_tenper_minus_base_OND2019_polar_ster_proj_mack_tiles.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove temporary frames
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

GIF saved to theta_tenper_minus_base_OND2019_polar_ster_proj_tiles.gif
